# Advent of Code - Day 19

For heaven's sake. I lose my prolog-running machine, and suddenly all the problems are about writing parsers.

Haven't done this for a bit, but let's try writing a recursive descent parser...

## Part 1

In [1]:
import re

Rats, version I'm using doesn't have pyrsistent. Get ready for lots of copying...

In [2]:
def parse_rule(str_in):
    m=re.match('(\d+):(.+)', str_in)
    
    return (int(m.group(1)),
            [[int(r) if r.isnumeric() else r.strip('""')
              for r in rhs.split()]
             for rhs in m.group(2).split('|')])  

In [3]:
def parse_input(str_in):
    
    [grammar, inputs]=str_in.split('\n\n')
    
    
    return ({lhs:rhs for (lhs, rhs) in 
             [parse_rule(nl) for nl in grammar.splitlines()]},
            
            [list(nl) for nl in inputs.splitlines()])

In [4]:
(grammar, inputs_ls)=parse_input(open('data/day19_test').read())
grammar

{0: [[4, 1, 5]],
 1: [[2, 3], [3, 2]],
 2: [[4, 4], [5, 5]],
 3: [[4, 5], [5, 4]],
 4: [['a']],
 5: [['b']]}

In [5]:
inputs_ls

[['a', 'b', 'a', 'b', 'b', 'b'],
 ['b', 'a', 'b', 'a', 'b', 'a'],
 ['a', 'b', 'b', 'b', 'a', 'b'],
 ['a', 'a', 'a', 'b', 'b', 'b'],
 ['a', 'a', 'a', 'a', 'b', 'b', 'b']]

In [6]:
def build_agenda(start, input_tokens, grammar):
    return [([start], input_tokens)]

In [7]:
def next_states(state_in, grammar):

    (tree, tokens)=state_in
    next_states=[]
    
    if not (tree and tokens):
        return next_states
    
    if tree[0]==tokens[0]:
            next_states.append((tree[1:].copy(),
                                tokens[1:].copy()))
    
    if isinstance(tree[0], int):
        for rhs in grammar[tree[0]]:
            rhscopy=rhs.copy()
            rhscopy.extend(tree[1:].copy())
            next_states.append((rhscopy, tokens.copy()))
    
    return next_states
        
        

In [8]:
def final_state(state_in):
    
    return state_in==([], [])
    

In [9]:
def recognise(tokens_ls, grammar):

    agenda=build_agenda(0, tokens_ls, grammar)
    
    while agenda:
        
        state=agenda.pop(0)
        
        if final_state(state):
            return True
        
        agenda[0:0]=next_states(state, grammar)
    
    return False

In [10]:
def day19_part1(file_in):
    
    (grammar, inputs_ls)=parse_input(open(file_in).read())
    
    return [tokens_ls for tokens_ls in inputs_ls
            if recognise(tokens_ls, grammar)]

In [11]:
assert len(day19_part1('data/day19_test')) == 2

In [12]:
len(day19_part1('data/day19_input'))

134

## Part 2

For a horrible moment, I thought I was going to have to convert my parser to run bottom up. But no! The rules aren't left recursive, so should be a problem.

Finally, get some benefits from doing Part 1 properly:

In [13]:
assert len(day19_part1('data/day19_testb'))==3

In [14]:
assert len(day19_part1('data/day19_testc'))==12

In [15]:
len(day19_part1('data/day19_inputb'))

377

Done!